In [1]:
#######TF API
# Shuffling datasets and randomizing Cali Data
import tensorflow as tf 
import pandas as pd

In [2]:
#Uploading California Mortgage data 
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()
# print(housing)

In [3]:
#No labels 
housing_d=housing["data"]
#Storing the labels
housing_L=housing["target"]
housing_feature_names=housing["feature_names"]

#Converting into dataframe
housing_df=pd.DataFrame(housing_d,columns=housing_feature_names)


In [4]:
#To tensor
# Convert the data to TensorFlow tensors
features_tf = tf.constant(housing_L, dtype=tf.float32)
labels_tf = tf.constant(housing_d, dtype=tf.float32)

# Combine the features and labels into a TensorFlow dataset
housing_tf_dataset = tf.data.Dataset.from_tensor_slices((features_tf))

In [5]:
###Creating Multiple datasets for training and validation and saving them as CSV
import sklearn as sk
from sklearn.model_selection import train_test_split
import os
directory="C:/Users/sergi/Documents/Py/"
os.chdir(directory)

train_filepaths=[]
validation_filepaths=[]
testing_filepaths=[]
for i in range(1,20):
    housing_df_shuffled = housing_df.sample(frac=1).reset_index(drop=True)
    X_train, X_temp, y_train, y_temp = train_test_split(housing_df_shuffled, housing_L, test_size=0.2)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5)
    train = pd.DataFrame(X_train, columns=housing_feature_names)
    train["MedHouseValue"] = y_train
    test = pd.DataFrame(X_test, columns=housing_feature_names)
    test["MedHouseValue"] = y_test
    val = pd.DataFrame(X_valid, columns=housing_feature_names)
    val["MedHouseValue"] = y_valid
    trainpath=str(directory)+"/calidata/training"+str(i)+".csv"
    valpath=str(directory)+"/calidata/validation"+str(i)+".csv"
    testingpath=str(directory)+"/calidata/testing"+str(i)+".csv"
    train.to_csv(trainpath, index=False)
    val.to_csv(valpath, index=False)
    test.to_csv(testingpath, index=False)
    train_filepaths.append(trainpath)
    testing_filepaths.append(testingpath)
    validation_filepaths.append(valpath)
# print(train_filepaths)
    

In [6]:
###Create a dataset that only contains training sets paths
filepath_dataset=tf.data.Dataset.list_files(train_filepaths, seed=42) ##This will return a dataset with shuffled datapaths
for i in filepath_dataset: 
    print(i) 

tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training7.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training9.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training10.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training2.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training16.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training12.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training4.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training8.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training1.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\sergi\\Documents\\Py\\\\calidata\\training5.csv', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\serg

In [7]:
##interleave: This is a function from tf API that reads from X files of the File_path_dataset at a time and interlines the lines
n_readers=5 #nr of files read at a time
dataset= filepath_dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers) 
#we are loading 5 random dataset at the same time, skippin the first row in all of them.
#If we ask to print this, it will show the first the first line of each of the 5 datasets, and will keeep on doing that until the end of the datasets 
#It is optimal to have datasets of the same length 
## Being specific. For each of the 5 datasets it will pull the lamna duntion at a time. It is secuential uless we specify the num_parallel_calls  -- This argument makes possible parallel reading


In [8]:
##interleave with parallel reading
dataset= filepath_dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers,num_parallel_calls=5 ) 


In [9]:
##Exanining the interleaved dataset 
for i in dataset.take(5): 
    print(i)
##showing the first line of the first 5 datasets 

tf.Tensor(b'3.6995,38.0,5.288503253796096,0.9587852494577006,1308.0,2.8373101952277655,34.1,-118.06,3.429', shape=(), dtype=string)
tf.Tensor(b'2.1806,20.0,2.6335540838852096,1.0022075055187638,1880.0,4.150110375275938,34.04,-118.28,2.788', shape=(), dtype=string)
tf.Tensor(b'3.07,15.0,5.385624344176286,0.9758656873032528,4591.0,2.408709338929696,39.76,-121.89,2.213', shape=(), dtype=string)
tf.Tensor(b'3.6523,28.0,5.551546391752577,1.0618556701030928,2086.0,2.6881443298969074,38.02,-122.1,0.867', shape=(), dtype=string)
tf.Tensor(b'5.1696,52.0,6.115902964959568,1.0242587601078168,1004.0,2.706199460916442,37.89,-122.29,0.375', shape=(), dtype=string)


In [10]:
import numpy as np
##Computing the SD and mean of each of the training sets features 
X_mean=[]
X_std=[]

for i in train_filepaths:
    mean=[]
    std=[]
    X= pd.read_csv(str(i))
    for j in X:
        mean.append(np.mean(X[j]))
        std.append(np.std(X[j]))
    X_mean.append(mean)
    X_std.append(std)

X_mean=pd.DataFrame(X_mean)[range(0,8)] # we are deselecting the output
X_std=pd.DataFrame(X_std)[range(0,8)] # we are deselecting the output

X_mean=tf.convert_to_tensor([np.mean(X_mean[i]) for i in X_mean],dtype=tf.float32) ## Converting into tensor
X_std=tf.constant([np.mean(X_std[i]) for i in X_std],dtype=tf.float32) ##Converting into tensor , both functions work the same

In [11]:
#From Byte string tensor to a standardise float tensor for every line of the interleave dataset
input_features_number= len(X_mean) # The number of features in the california housing dataset minust the target

#Transfrtomning into a float tensor

def parse_csv_line(line):
    defs = [0.] * input_features_number + [0.]  # Default values
    fields = tf.io.decode_csv(line, record_defaults=defs)
    features = tf.stack(fields[:-1])## input variables are all but the last element
    label = tf.stack(fields[-1:])## y variable is in the last element of the row
    return features, label

def preprocess(line):
    x, y = parse_csv_line(line)
    x = (x - X_mean) / X_std  # Normalize features
    return x, y

def preprocess_out(line):
    x, y = parse_csv_line(line)
    return y

##testing this 

preprocess(b'6.1359,16.0,4.0,0.8,45.0,1.8,37.96,-122.5,2.369')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 1.1891571 , -1.0038002 , -0.57199955, -0.61822224, -1.2149907 ,
        -0.12796491,  1.0883933 , -1.4608583 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.369], dtype=float32)>)

In [12]:
## Loading random data and converting into tebsor float-- Putting everything together. 
# This final step will give us a standardised, preprocessed X training data where each row of input features is a float 32byte tensor
# This will be a general function that loads multiple csv files from california datset, preprocess them, shuffles them and batch them
# Prefetch is important because it tells the  training algoriths that while is performing a batch training, to go one step ahead and retrieve the data for the next batch
# Prefetch increases performance dramatically

def csv_reader_dataset(filepaths, n_readers=5, n_parse_threads=5, shuffle_buffer_size=10_000, seed=42, batch_size=30):
    dataset = tf.data.Dataset.list_files(filepaths, seed=seed)
    dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_parse_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size, seed=seed)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)  # Use AUTOTUNE for prefetching
    return dataset



In [13]:
#Preprocessing data for validation, training and testing sets
train_set=csv_reader_dataset(train_filepaths)
test_set=csv_reader_dataset(testing_filepaths)
valid_set=csv_reader_dataset(validation_filepaths)

In [14]:
## Model fitting: Keras
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),

    tf.keras.layers.Dense(1)  # Output layer
])

# Compile the model
model.compile(loss="mse", optimizer="nadam")

# Fit the model on the training dataset
history = model.fit(train_set,  # The training dataset
                    validation_data=valid_set,  # Validation dataset
                    epochs=10)  # Number of epochs

# Evaluate the model on the test dataset
test_mse = model.evaluate(test_set)  # Test dataset
print("Test MSE:", test_mse)

Epoch 1/10
  10409/Unknown 7s 593us/step - loss: 1.4827

C:\Users\sergi\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


10458/10458 ━━━━━━━━━━━━━━━━━━━━ 8s 665us/step - loss: 1.4822 - val_loss: 1.3556
Epoch 2/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 7s 700us/step - loss: 1.3383 - val_loss: 1.3509
Epoch 3/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 10s 955us/step - loss: 1.3365 - val_loss: 1.3515
Epoch 4/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 1.3329 - val_loss: 1.3531
Epoch 5/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 11s 999us/step - loss: 1.3313 - val_loss: 1.3486
Epoch 6/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 1.3332 - val_loss: 1.3486
Epoch 7/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 1.3342 - val_loss: 1.3484
Epoch 8/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 1.3362 - val_loss: 1.3488
Epoch 9/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 1.3334 - val_loss: 1.3492
Epoch 10/10
10458/10458 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 1.3343 - val_loss: 1.3484
1308/1308 ━━━━━━━━━━━━━━━━━━━━ 1s 688us/step - loss: 1.3206
Test MSE: 1.3278415203094482


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 30)                  │             270 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,696 (14.44 KB)

 Trainable params: 1,231 (4.81 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,465 (9.63 KB)

In [16]:
predictions = model.predict(train_set)

10458/10458 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step
